In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras import layers
from keras import backend as K
from keras import regularizers
from keras.constraints import max_norm
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.models import Model
from keras.initializers import glorot_uniform
from keras.layers import Input,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D,AveragePooling2D,MaxPooling2D,Dropout
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [2]:
# Import data
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
train_features = train.drop(['target', 'ID_code'], axis=1)
train_targets = train['target']
test_features = test.drop(['ID_code'], axis=1)

In [4]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(train_features.values)
y = train_targets.values
#X_test = sc.transform(X_test)
#test_features = sc.transform(test_features)

In [5]:
# Training & test stratified split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify = y, test_size = 0.10)

In [6]:
# Reshape
X_train = np.reshape(X_train,(180000,10,10,2))
X_valid = np.reshape(X_valid,(20000,10,10,2))
print(X_train.shape)
print(X_valid.shape)

(180000, 10, 10, 2)
(20000, 10, 10, 2)


In [7]:
# Model training        
def Convnet(input_shape = (10,10,2),classes = 1):

    X_input = Input(input_shape)
 
    # Stage 1
    X = Conv2D(64,kernel_size = (3,3),strides = (1,1),name = "conv1", kernel_initializer = glorot_uniform(seed = 0))(X_input)
    #X = BatchNormalization()(X)
    X = Activation("tanh")(X)
    x = Dropout(rate=0.2)(X)
    # Stage 2
    X = Conv2D(128,kernel_size=(2,2),strides=(1,1),name="conv2",kernel_initializer=glorot_uniform(seed=0))(x)
    #X = BatchNormalization()(X)
    X = Activation("tanh")(X)
    x = Dropout(rate=0.1)(X)
    X = Conv2D(128,kernel_size=(3,3),strides=(2,2),name="conv3",kernel_initializer=glorot_uniform(seed=0))(x)
    #X = BatchNormalization()(X)
    X = Activation("tanh")(X)
    x = Dropout(rate=0.1)(X)
    X = Conv2D(256,kernel_size=(2,2),strides=(1,1),name="conv4",kernel_initializer=glorot_uniform(seed=0))(x)
    #X = BatchNormalization()(X)
    X = Activation("tanh")(X)
    x = Dropout(rate=0.1)(X)
    # Stage 3
    X = Conv2D(64,kernel_size=(2,2),strides=(2,2),name="conv5",kernel_initializer=glorot_uniform(seed=0))(x)
    #X = BatchNormalization()(X)
    X = Activation("tanh")(X)
 
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid')(X)
 

    model = Model(inputs=X_input,outputs=X)
 
    return model

In [8]:
model = Convnet()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
earlystop = EarlyStopping(monitor='val_loss', mode='auto', patience=10, verbose=1)

In [10]:
history = model.fit(X_train, y_train, epochs=300, batch_size=50000, validation_data=(X_valid, y_valid), callbacks=[earlystop])

Train on 180000 samples, validate on 20000 samples
Epoch 1/300
180000/180000 [==============================] - 25s 137us/step - loss: 0.6585 - acc: 0.5945 - val_loss: 0.4862 - val_acc: 0.8336
Epoch 2/300
180000/180000 [==============================] - 9s 51us/step - loss: 0.3824 - acc: 0.8723 - val_loss: 0.3842 - val_acc: 0.8995
Epoch 3/300
180000/180000 [==============================] - 9s 51us/step - loss: 0.3359 - acc: 0.8970 - val_loss: 0.2644 - val_acc: 0.9021
Epoch 4/300
180000/180000 [==============================] - 9s 51us/step - loss: 0.2830 - acc: 0.8938 - val_loss: 0.2555 - val_acc: 0.9068
Epoch 5/300
180000/180000 [==============================] - 9s 51us/step - loss: 0.2641 - acc: 0.9046 - val_loss: 0.2538 - val_acc: 0.9068
Epoch 6/300
180000/180000 [==============================] - 9s 51us/step - loss: 0.2546 - acc: 0.9071 - val_loss: 0.2427 - val_acc: 0.9089
Epoch 7/300
180000/180000 [==============================] - 9s 51us/step - loss: 0.2487 - acc: 0.9079 - va

In [11]:
#--AUC
y_pred_keras = model.predict(X_valid).ravel()

In [12]:
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_valid, y_pred_keras)
auc_keras = auc(fpr_keras, tpr_keras)

In [13]:
auc_keras

0.8646032621771631

In [14]:
roc_auc_score(y_valid, y_pred_keras)

0.8646032621771631

In [15]:
X_test = sc.transform(test_features.values)
X_test = np.reshape(X_test,(200000,10,10,2)) 
prediction = model.predict(X_test)

In [16]:
result = pd.DataFrame({"ID_code": test.ID_code.values})
result["target"] = prediction
result.to_csv("submission.csv", index=False)

In [17]:
model.save('./cnn_model1.h5')

In [18]:
# Save model architecture
with open('cnn_model1_architecture.json', 'w') as f:
    f.write(model.to_json())